In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib3
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from requests.packages.urllib3.exceptions import InsecureRequestWarning
# 禁用安全请求警告
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
# 获取所有的url
def getUrls(url, pageSum):
    urls = []
    pns = [str(i * 1) for i in range(11,pageSum+1)]
    ul = url.split('=')
    for pn in pns:
        ul[-1] = pn
        url = '='.join(ul)
        urls.append(url)
    return urls

In [ ]:
# 获取标签为空处理
def just_null(tag):
    if tag is not None:
        return tag.get_text().strip()
    else:
        return '未知'
    
# 解析标签内容
def analysis_data(page,data_list):
    for i,tag in enumerate(data_list):
        print ('第 ',page,' 页 ',' 第 ',i,' 条')
        
        # 职位信息
        job_info = tag.find('div', attrs={'class': 'job-info'})
        job_type = just_null(tag.find('i', attrs={}).find('b', attrs={}))
        title = just_null(job_info.find('h3', attrs={}).find('a', attrs={}))
        compensation = just_null(job_info.find('span', attrs={'class': 'text-warning'}))
        city = just_null(job_info.find('a', attrs={'class': 'area'}))
        education = just_null(job_info.find('span', attrs={'class': 'edu'}))
        work_experience = just_null(job_info.find('span', attrs={'class': ''}))
        
        time = job_info.find('p', attrs={'class': 'time-info clearfix'}).find('time', attrs={}).get('title')
        # 公司信息
        company_info = tag.find('div', attrs={'class': 'company-info nohover'})
        company = just_null(company_info.find('p', attrs={'class': 'company-name'}).find('a', attrs={}))
        company_type = just_null(company_info.find('a', attrs={'class': 'industry-link'}))
        company_link = job_info.find('h3', attrs={}).find('a', attrs={}).get('href')
        feedback_time = just_null(job_info.find('p', attrs={'class': 'time-info clearfix'}).find('span', attrs={}))

        print (job_type,'   ',title,'   ',company)
        
        # 获取公司人数
        if job_type == '猎':
            company_number = '未知'
        else:
            print (company_link)
            http2 = urllib3.PoolManager()
            r2 = http2.request('GET',company_link)
            soup2 = BeautifulSoup(r2.data, 'lxml')
            info_list = soup2.find('ul', attrs={'class':'new-compintro'}).find_all('li', attrs={})
            if len(info_list) < 2:
                company_number = '未知'
            else:
                company_number = just_null(info_list[1])[5:]
        df.loc[len(df.index)+1] = [title,job_type,compensation,city,education,work_experience,feedback_time,time,company,company_type,company_number,company_link]

In [ ]:
# base_url = 'https://www.liepin.com/zhaopin/?fromSearchBtn=2&ckid=0c3548af3dfd5fb9&d_=&degradeFlag=0&init=-1&imscid=R000000058&key=python&headckid=0c3548af3dfd5fb9&d_pageSize=40&siTag=I-7rQ0e90mv8a37po7dV3Q%7EfA9rXquZc5IkJpXC-Ycixw&d_headId=9133450e89c4334ba542648a1889de2c&d_ckId=9133450e89c4334ba542648a1889de2c&d_sfrom=search_fp_bar&d_&curPage='
# bsse_url = 'https://www.liepin.com/zhaopin/?fromSearchBtn=2&ckid=0c3548af3dfd5fb9&d_=&degradeFlag=0&init=-1&imscid=R000000058&key=ios&headckid=0c3548af3dfd5fb9&d_pageSize=40&siTag=I-7rQ0e90mv8a37po7dV3Q%7EfA9rXquZc5IkJpXC-Ycixw&d_headId=9133450e89c4334ba542648a1889de2c&d_ckId=9133450e89c4334ba542648a1889de2c&d_sfrom=search_fp_bar&d_&curPage='
base_url = 'https://www.liepin.com/zhaopin/?ckid=eee75740ae6b9bd9&fromSearchBtn=2&degradeFlag=0&init=-1&key=ios&imscid=R000000058&headckid=eee75740ae6b9bd9&d_pageSize=40&siTag=njBNTo3xt0z6AJkTGYQoqw%7EfA9rXquZc5IkJpXC-Ycixw&d_headId=970a437ae1072c6d0bb02bb3aa32b573&d_ckId=970a437ae1072c6d0bb02bb3aa32b573&d_sfrom=search_fp_bar&d_&curPage='
# 获取所有的url
all_url = getUrls(base_url,21)
# all_url = ['https://www.liepin.com/zhaopin/?fromSearchBtn=2&ckid=0c3548af3dfd5fb9&d_=&degradeFlag=0&init=-1&imscid=R000000058&key=python&headckid=0c3548af3dfd5fb9&d_pageSize=40&siTag=I-7rQ0e90mv8a37po7dV3Q%7EfA9rXquZc5IkJpXC-Ycixw&d_headId=9133450e89c4334ba542648a1889de2c&d_ckId=9133450e89c4334ba542648a1889de2c&d_sfrom=search_fp_bar&d_&curPage=1']
# 创建表
df = pd.DataFrame(columns=['title','job_type','compensation','city','education','work_experience','feedback_time','time','company','company_type','company_number','company_link',])
# 开始抓取
for i,url in enumerate(all_url):
    http = urllib3.PoolManager()
    r = http.request('GET',url)
    soup = BeautifulSoup(r.data, 'lxml')
    data_list = soup.find('ul', attrs={'class': 'sojob-list'}).find_all('li', attrs={})
    analysis_data(i,data_list)

In [37]:
df

,title,job_type,compensation,city,education,work_experience,feedback_time,time,company,company_type,company_number,company_link
1,iOS开发工程师（中高级）,企,24-48万,上海,统招本科,2年工作经验,投递后：5个工作日内反馈,2017年12月21日,喜马拉雅FM,互联网/移动互联网/电子商务,1000-2000人,https://www.liepin.com/job/1910754019.shtml
2,高级IOS工程师,企,面议,北京,统招本科,3年工作经验,投递后：5个工作日内反馈,2017年12月21日,vipkid,互联网/移动互联网/电子商务,2000-5000人,https://www.liepin.com/job/198897917.shtml
3,QA工程师（Android/iOS应用测试）,企,30-60万,北京-朝阳区,本科及以上,3年工作经验,投递后：5个工作日内反馈,2017年12月21日,美团点评,互联网/移动互联网/电子商务,10000人以上,https://www.liepin.com/job/199180203.shtml
4,IOS软件开发工程师,企,面议,上海-长宁区,本科及以上,3年工作经验,投递后：5个工作日内反馈,2017年12月22日,京东,互联网/移动互联网/电子商务,10000人以上,https://www.liepin.com/job/1910910707.shtml
5,资深iOS开发工程师,企,面议,武汉-武昌区,本科及以上,3年工作经验,未知,2017年12月21日,今日头条,互联网/移动互联网/电子商务,1000-2000人,https://www.liepin.com/job/1911241165.shtml
6,移动端开发主管（ios/android）,企,36-54万,上海,统招本科,7年工作经验,投递后：5个工作日内反馈,2017年12月21日,跃橙信息科技(上海)有限公司,互联网/移动互联网/电子商务,50-99人,https://www.liepin.com/job/1910994152.shtml
7,资深ios开发工程师,企,32-56万,北京,本科及以上,经验不限,投递后：5个工作日内反馈,2017年12月21日,高德软件,互联网/移动互联网/电子商务,2000-5000人,https://www.liepin.com/job/1911434256.shtml
8,iOS开发工程师,企,11-21万,上海,本科及以上,1年工作经验,投递后：10个工作日内反馈,2017年12月21日,纯米科技,互联网/移动互联网/电子商务,100-499人,https://www.liepin.com/job/1911432662.shtml
9,iOS研发工程师,企,15-30万,上海,统招本科,3年工作经验,投递后：2个工作日内反馈,2017年12月21日,上海见寻文化创意有限公司,互联网/移动互联网/电子商务,1-49人,https://www.liepin.com/job/1911431018.shtml
10,IOS R & D Engineer,企,18-36万,北京,学历不限,经验不限,投递后：5个工作日内反馈,2017年12月21日,北京布劳克晨信息技术有限公司,互联网/移动互联网/电子商务,50-99人,https://www.liepin.com/job/1911429064.shtml


In [38]:
df.to_csv('ios2.csv')

In [294]:
len(df)

206